In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from utils import *

# helps in text preprocessing
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# helps in model building
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.callbacks import EarlyStopping

%matplotlib inline

--ip=127.0.0.1


In [2]:
parent_dir = os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

In [3]:

df_train = pd.read_csv(os.path.join(parent_dir, 'Data\AugmentedData\english_train_augmented.csv'))
df_dev = pd.read_csv(os.path.join(parent_dir, 'Data\PreprocessedData\english_dev_preprocess.csv'))
df_test = pd.read_csv(os.path.join(parent_dir, 'Data\PreprocessedData\english_test_preprocess.csv'))

In [4]:
df_train = df_train[['augmented_text', 'label']]
df_dev = df_dev[['preprocessed_text', 'label']]
df_test = df_test[['preprocessed_text', 'label']]

In [5]:
label_replacement = {
    'Hope_speech': 0,
    'Non_hope_speech': 1,
    'not-English': 2,
}

df_train['label'] = df_train['label'].replace(label_replacement)
df_test['label'] = df_test['label'].replace(label_replacement)
df_dev['label'] = df_dev['label'].replace(label_replacement)

# Drop rows with label 2
df_train = df_train[df_train['label'] != 2]
df_test = df_test[df_test['label'] != 2]
df_dev = df_dev[df_dev['label'] != 2]

In [6]:
X_train, y_train = df_train['augmented_text'].to_numpy(), df_train['label'].to_numpy()
X_dev, y_dev = df_dev['preprocessed_text'].to_numpy(), df_dev['label'].to_numpy()
X_test, y_test = df_test['preprocessed_text'].to_numpy(), df_test['label'].to_numpy()

In [7]:
t = Tokenizer()
t.fit_on_texts(X_train)

In [8]:
encoded_train = t.texts_to_sequences(X_train)
encoded_dev = t.texts_to_sequences(X_dev)
encoded_test = t.texts_to_sequences(X_test)

In [9]:
max_length = 8
padded_train = pad_sequences(encoded_train, maxlen=max_length, padding='post')
padded_dev = pad_sequences(encoded_dev, maxlen=max_length, padding='post')
padded_test = pad_sequences(encoded_test, maxlen=max_length, padding='post')

In [10]:
vocab_size = len(t.word_index) + 1

# define the model
model = Sequential()
model.add(Embedding(vocab_size, 24, input_length=max_length))
model.add(SimpleRNN(24, return_sequences=False))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# summarize the model
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 8, 24)             521064    
                                                                 
 simple_rnn (SimpleRNN)      (None, 24)                1176      
                                                                 
 dense (Dense)               (None, 1)                 25        
                                                                 
Total params: 522,265
Trainable params: 522,265
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

# fit the model
model.fit(
    x=padded_train,
    y=y_train,
    epochs=100,
    validation_data=(padded_dev, y_dev), verbose=1,
    callbacks=[early_stop]
)

Epoch 1/100
1324/1324 [==============================] - 6s 3ms/step - loss: 0.4508 - accuracy: 0.7819 - val_loss: 0.3840 - val_accuracy: 0.8374
Epoch 2/100
1324/1324 [==============================] - 5s 4ms/step - loss: 0.2913 - accuracy: 0.8773 - val_loss: 0.4171 - val_accuracy: 0.8296
Epoch 3/100
1324/1324 [==============================] - 5s 4ms/step - loss: 0.2297 - accuracy: 0.9076 - val_loss: 0.4101 - val_accuracy: 0.8391
Epoch 4/100
1324/1324 [==============================] - 5s 4ms/step - loss: 0.1985 - accuracy: 0.9212 - val_loss: 0.3760 - val_accuracy: 0.8539
Epoch 5/100
1324/1324 [==============================] - 5s 4ms/step - loss: 0.1782 - accuracy: 0.9314 - val_loss: 0.4081 - val_accuracy: 0.8479
Epoch 6/100
1324/1324 [==============================] - 5s 4ms/step - loss: 0.1658 - accuracy: 0.9378 - val_loss: 0.4207 - val_accuracy: 0.8370
Epoch 7/100
1324/1324 [==============================] - 5s 4ms/step - loss: 0.1556 - accuracy: 0.9411 - val_loss: 0.3921 - val_ac

In [12]:
train_preds = model.predict(padded_train)
dev_preds = model.predict(padded_dev)
test_preds = model.predict(padded_test)

train_preds = np.where(train_preds > 0.5, 1, 0)
dev_preds = np.where(dev_preds > 0.5, 1, 0)
test_preds = np.where(test_preds > 0.5, 1, 0)

train_preds = train_preds.flatten()
dev_preds = dev_preds.flatten()
test_preds = test_preds.flatten()

89/89 [==============================] - 0s 2ms/step


In [14]:
computeAllScores(train_preds, dev_preds, test_preds, True, True)

Accuracy Train:  0.9521246458923512
Accuracy Dev:  0.8514607532558958
Accuracy Test:  0.8519169890960253
Weighted F1 Train:  0.9521278260542252
Weighted F1 Dev:  0.8618461949325626
Weighted F1 Test:  0.8665794297608288
Macro F1 Train:  0.9521147809004158
Macro F1 Dev:  0.6324677379007931
Macro F1 Test:  0.6328780795728404
Micro F1 Train:  0.9521246458923512
Micro F1 Dev:  0.8514607532558958
Micro F1 Test:  0.8519169890960253
Weighted Recall Train:  0.9521246458923512
Weighted Recall Dev:  0.8514607532558958
Weighted Recall Test:  0.8519169890960253
Macro Recall Train:  0.9521952457520335
Macro Recall Dev:  0.6565333558491516
Macro Recall Test:  0.6712371770150405
Micro Recall Train:  0.9521246458923512
Micro Recall Dev:  0.8514607532558958
Micro Recall Test:  0.8519169890960253
Confusion Matrix Train: 
[[20470  1112]
 [  916 19862]]
Confusion Matrix Dev: 
[[ 113  159]
 [ 263 2306]]
Confusion Matrix Test: 
[[ 113  137]
 [ 284 2309]]
